### <b>SQL</b> 

__По таблице default.peas определим количество усердных учеников образовательных курсов.__ 
________________________

In [1]:
import pandahouse as ph

In [3]:
# подключение к database clickhouse
connection_default = {'host' : 'http://clickhouse.lab.karpov.courses',
                      'database'  : 'default',
                      'user' :'student', 
                      'password' : 'dpo_python_2020'
                     }

# Запрос определения усердных учеников
task_1 = '''
SELECT
    COUNT(DISTINCT(st_id)) as assiduous_student
FROM(
    SELECT 
        st_id, 
        SUM(correct) as correctly_solved
    FROM peas
    WHERE MONTH(timest) = (SELECT MAX(MONTH(timest)) as timest FROM peas)
    GROUP BY st_id
    HAVING correctly_solved >= 20)
'''
task_2_1 = ph.read_clickhouse(query=task_1, connection=connection_default)
task_2_1

,assiduous_student
0,136


__Количество решений заданий в день на образовательной платформе ограничено, чтобы получить доступ к неограниченному количеству заданий в определенной дисциплине, необходимо приобрести подписку. Наша команда провела эксперимент, где был протестирован новый экран оплаты.__

__Для анализа нашего эксперемента рассмотрим метрики тестовой и контрольной группы:__ 
________________________

 - ARPU 
 - ARPAU 
 - CR в покупку 
 - СR активного пользователя в покупку 
 
 Активным будем считать пользователя, который за все время решил больше 10 задач правильно в любых дисциплинах.
 - CR пользователя из активности по математике в покупку курса по математике
 
 Активным по математики будем считать пользователя, который за все время решил больше 2 и более задач правильно по математике.
 - ARPU считается относительно всех пользователей, попавших в группы.

In [4]:
# Запрос определения метрик:
task_2 = '''
SELECT f.test_grp,
       ROUND(sum_all_money/count_all, 2) ARPU, -- выводим ARPU
       ROUND(sum_active_money/count_active_all, 2) ARPAU, -- выводим ARPAU
       ROUND(count_all_payed*100/count_all, 2) CR, -- выводим CR в покупку
       ROUND(count_all_payed_active*100/count_active_all, 2) CR_active, -- выводим CR активного пользователя в покупку
       ROUND(count_all_payed_active_math*100/count_active_math, 2) CR_active_math -- выводим CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике
FROM
    (
    SELECT s.test_grp AS test_grp, -- подсчитываем нужные для рассчёта значения
           COUNT(*) count_all,
           COUNT(IF(c.check=1, 1, null)) count_all_payed,
           COUNT(IF(c.check=1, p.count_active, null)) count_all_payed_active,
           COUNT(IF(c.check=1, p.count_active_math, null)) count_all_payed_active_math,
           SUM(c.sum_money) sum_all_money,
           SUM(IF(p.count_active=1,c.sum_money,0)) sum_active_money,
           SUM(c.count_math_payed) count_math_payed,
           SUM(c.sum_math_payed) sum_math_money,
           SUM(p.count_active) count_active_all,
           SUM(p.count_active_math) count_active_math
    FROM studs s
    LEFT JOIN -- присоединяем таблицу final_project_check с информацией покупки
            (
            SELECT c.st_id,
                   1 check, -- "1" совершена покупка
                   SUM(c.money) sum_money,
                   MAX(IF(c.subject = 'Math', 1, 0)) count_math_payed,
                   SUM(IF(c.subject = 'Math', c.money, 0)) sum_math_payed
                  
            FROM final_project_check c
            GROUP BY c.st_id
            ) c
    ON (c.st_id = s.st_id)
    LEFT JOIN  -- присоединяем таблицу peas с информацией решений заданий
            (
            SELECT p.st_id,
                   1 check,-- "1" совершена покупка
                   IF(COUNT(*)>10,1,NULL) count_active, -- отбираем активных (решивших больше 10 задач правильно в любых дисциплинах)            
                   IF(COUNT(IF(p.subject = 'Math', 1, NULL)) >= 2, 1, NULL) count_active_math -- отбираем активных по математике (решивших >= 2 задач правильно по математике)
            FROM peas p
            WHERE p.correct = 1 -- условие, что задача решена правильно
            GROUP BY p.st_id
            ) p 
    ON (p.st_id = s.st_id)
    GROUP BY s.test_grp
    ) f
'''
task_2_2 = ph.read_clickhouse(query=task_2, connection=connection_default)
task_2_2

,test_grp,ARPU,ARPAU,CR,CR_active,CR_active_math
0,control,4540.98,10393.70,4.92,11.02,10.20
1,pilot,11508.47,29739.58,10.85,26.04,14.29
